# MLP多层感知器的使用
多层感知器常用来做分类，效果非常好，比如文本分类，效果比SVM，贝叶斯好多了
多层感知器的介绍

![](http://static.oschina.net/uploads/space/2016/0421/214930_jdyJ_937000.png)

MLP（多层感知器）神经网络是常见的ANN算法，它由一个输入层，一个输出层和一个或多个隐藏层组成。

在MLP中的所有神经元都差不多，每个神经元都有几个输入（连接前一层）神经元和输出（连接后一层）神经元，该神经元会将相同值传递给与之相连的多个输出神经元

一个神经网络训练网将一个特征向量作为输入，将该向量传递到隐藏层，然后通过权重和激励函数来计算结果，并将结果传递给下一层，直到最后传递给输出层才结束
    

## 使用MLP进行手写识别

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

In [4]:
# Network Parameters
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [6]:
# create model
def multilayer_perceptron(_X, _weights, _biases):
    layer1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1']))
    layer2 = tf.nn.sigmoid(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2']))
    return tf.matmul(layer2, _weights['out']) + _biases['out']

In [7]:
# store layers weight and bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [8]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [9]:
# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
# init
init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys})/total_batch
        if epoch % display_step == 0:
            print("Epoch: ,", '%04d' % (epoch+1), "cost= ", "{:.9f}".format(avg_cost))
            
    print("Optimization Finished!")
    
    #Test 
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    #Calculate
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy: ", accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

Epoch: , 0001 cost=  1.752671735
Epoch: , 0002 cost=  0.528026813
Epoch: , 0003 cost=  0.350073692
Epoch: , 0004 cost=  0.256389080
Epoch: , 0005 cost=  0.193637855
Epoch: , 0006 cost=  0.151259659
Epoch: , 0007 cost=  0.117683001
Epoch: , 0008 cost=  0.091971789
Epoch: , 0009 cost=  0.072809689
Epoch: , 0010 cost=  0.057189651
Epoch: , 0011 cost=  0.044978852
Epoch: , 0012 cost=  0.035099575
Epoch: , 0013 cost=  0.027744376
Epoch: , 0014 cost=  0.021740625
Epoch: , 0015 cost=  0.016794334
Optimization Finished!
Accuracy:  0.94350016
